In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from tqdm import trange
import time
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.DataFrame(columns=['INDEX','LINK','LINK_RTF'])

In [5]:
root_url = 'https://www.corteconstitucional.gov.co'

In [6]:
for i in tqdm(range(51)):
    
    try:
        page_url = '/relatoria/providencia.aspx?pg='+ str(i) +'&buscar=&anio=&total=35718'
        main_url = root_url + page_url

        main_html_text = requests.get(main_url).text
        main_soup = BeautifulSoup(main_html_text, 'lxml')

        docs = main_soup.find_all('div', class_ = 'grow')

        for doc in docs:
            
            
            doc_link = doc.find('a')['href']

            inner_doc_url = root_url + doc_link
            
            try:
                
                html_inner_doc = requests.get(inner_doc_url).text
                inner_doc_soup = BeautifulSoup(html_inner_doc, 'lxml')
                doc_index = doc.find('a').text
                doc_rtf_link = root_url + (inner_doc_soup.find('a')['href'])

                new_row = {'INDEX':doc_index,'LINK':inner_doc_url,'LINK_RTF':doc_rtf_link}
                data = data.append(new_row, ignore_index=True)
            
            except Exception as e:
                
                print("Error!: " + str(e))
                print('Problema en: '+ inner_doc_url)
                print('Pagina: ' + main_url)
                print('Etapa:' + str(i))
                print('*'*30)
            

    except Exception as e:
        print("Error!: " + str(e))

 51%|█████     | 26/51 [28:47<24:39, 59.19s/it]

Error!: 'NoneType' object is not subscriptable
Problema en: https://www.corteconstitucional.gov.co/Relatoria/autos/2017/A025-17.htm
Pagina: https://www.corteconstitucional.gov.co/relatoria/providencia.aspx?pg=26&buscar=&anio=&total=35718
Etapa:26
******************************


100%|██████████| 51/51 [54:46<00:00, 64.45s/it]


In [7]:
data

,INDEX,LINK,LINK_RTF
0,A123-21,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
1,A121-21,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
2,A115-21,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
3,A107-21,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
4,A106-21,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
...,...,...,...
5144,A061-11,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5145,A060-11,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5146,A059-11,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...
5147,A058-11,https://www.corteconstitucional.gov.co/Relator...,https://www.corteconstitucional.gov.co/sentenc...


In [10]:
file_name = '/content/drive/My Drive/LINKS_1.csv'
data.to_csv(file_name, encoding='utf-8', index= False)